In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import datetime
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm

/Users/yanran/miniconda3/envs/hw2/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/yanran/miniconda3/envs/hw2/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /Users/yanran/miniconda3/envs/hw2/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <E459C462-F863-3A5A-AC9F-FD77B14BE845> /Users/yanran/miniconda3/envs/hw2/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/yanran/miniconda3/envs/hw2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. S

In [3]:
DEBUG = False
USE_DT_FILE = False
Test = True

In [4]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
train_data = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle=False, num_workers=2)

classes = ('Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck')

Files already downloaded and verified
Files already downloaded and verified


In [6]:
# device = torch.device('mps')
device = 'cpu'

In [7]:
import torchvision.models as models
vgg16 = models.vgg16(pretrained=False)
model_path = 'checkpoints/finetuned_vgg16_9'
vgg16.classifier[4] = nn.Linear(4096,1024)
vgg16.classifier[6] = nn.Linear(1024,10)
vgg16.load_state_dict(torch.load(model_path))
vgg16.to(device)

vgg16.eval()


/Users/yanran/miniconda3/envs/hw2/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/yanran/miniconda3/envs/hw2/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [8]:

import torch
import torch.nn as nn
import math
import copy
import random
import numpy as np
import struct
import torch.nn.init as init

# input : tensor output: binary string
def binary(num):
  return ''.join('{:0>8b}'.format(c) for c in struct.pack('!f', num))

#input: mantissa bitstring
#output: float value
def calc_mantissa(mantissa):
  res = 0
  for k in range(len(mantissa)):
      if mantissa[k] == '1':
        res += 2**(-k-1)
  return res

#input exp: bitstring
# new_exp_len: new length of exp
def calc_exp(exp, new_exp_len):
  limit = 2**(new_exp_len) - 1
  # if exp is more than new_exp_len limit, truncate to new_exp_len limit.
  bias = 2**(len(exp)-1) - 1
  val = int(exp,2) - bias
  if val > limit:
      return limit
  if val < -limit + 1:
      return -limit + 1
      pass
  return val


def round_fp8(x, exp = 4):
  '''
  Quantizes input tensor to FP8 data format
  inputs  x:      original tensor
          exp:    number of bits used for exponent field
                  e.g. E5M2 has 5 exp bits, E4M3 has 4 exp bits
  output  x_32:   quantized tensor
  '''

  x_fp8 = copy.deepcopy(x)


  result = 1.0
  bin_str = binary(x)

  bin_mantissa = bin_str[9:32]
  res_mantissa = bin_mantissa[:7-exp]
  result += calc_mantissa(res_mantissa)

  bin_exp = bin_str[1:9]
  exp_int = calc_exp(bin_exp, exp)
  result *= 2**exp_int

  if bin_str[0] == '1':
    result *= -1

  return result


def bisection_quantization(num, bits = 7):
  val = abs(num)
  inversed_bits = []
  # Bisection tree quantization
  range_min, range_max = 0, 1
  for _ in range(bits):
      mid = (range_min + range_max) / 2
      if val >= mid:
          inversed_bits.append(1)
          range_min = mid
      else:
          inversed_bits.append(0)
          range_max = mid

  quantized_val = 0
  for k, bit in enumerate(inversed_bits):
      if bit:
          quantized_val += 2**-(k + 1)

  return quantized_val

#the input is normalized tensor x,
def round_dt8(x, exp = 4):
  val = copy.deepcopy(x)
  num_levels = 2 ** (7)


  sign_bit = 0 if val >= 0 else 1
  val = abs(val)
  exp_bits = 0
  while val < 0.1:
      val *= 10
      exp_bits += 1

  bs_bits = max(0, 6 - exp_bits)
  exp_bits = min(7, exp_bits)

  if exp_bits == 0:
      quantized_val = bisection_quantization(val, 7)
  elif exp_bits >= 6:
      quantized_val = val
  else:
      quantized_val = bisection_quantization(val, bs_bits)

  quantized_val = quantized_val if sign_bit == 0 else -quantized_val
  quantized_val *= 10**(-exp_bits)
  return quantized_val


def quantize_rowwise(x: torch.Tensor, dt = False):
  abso = torch.abs(x)
  output_maxs  = torch.max(abso,1)[0].unsqueeze(-1)
  output = x  / output_maxs[None,:]
  if not dt:
      output.apply_(round_fp8)
  else:
      output.apply_(round_dt8)
  return torch.squeeze(output), output_maxs

def dequantize_rowwise(x: torch.Tensor, state_x: torch.Tensor):
  output = x * state_x
  return output

def init_weights(m, in_std = 1.0):
  if isinstance(m, nn.Linear):
      init.normal_(m.weight, mean=0.0, std=in_std)
      if m.bias is not None:
          init.zeros_(m.bias)

def measure_quantization_error(original_tensor, dequantized_tensor):
  abs_error = torch.abs(original_tensor - dequantized_tensor)
  return torch.mean(abs_error), abs_error

def quantize_stable_embedding(x, batch_size, dt = False):
  if (x.numel() % batch_size != 0):
    print("Invalid batch size. Batch size should be a divisor of " + str(x.numel()))
    return 

  flatarg = torch.argsort(x.flatten())
  indexing = flatarg.reshape((x.numel()//batch_size,batch_size))

  reshapedx = x.flatten()[indexing]
  output, maxes = quantize_rowwise(reshapedx,dt)

  return output.reshape(x.shape), maxes, indexing

def dequantize_stable_embedding(input, maxes, indexing):
  outreshape = input.reshape(indexing.shape)

  dequant = dequantize_rowwise(outreshape, maxes)
  return dequant.reshape(input.shape)

In [9]:
def quantize_stable_embedding(x, batch_size, dt = False):
  '''Qunatizes the given array
  Batch size must be a divisor of the array size
  Returns the quantized array, the maximums, and the indexes
  DT  = false, means using fp8
  Dt = true, means using the dynamic tree
  '''
  if (x.numel() % batch_size != 0):
    print("Invalid batch size. Batch size should be a divisor of " + str(x.numel()))
    return

  flatarg = torch.argsort(torch.abs(x.flatten()))
  indexing = flatarg.reshape((x.numel()//batch_size,batch_size))

  reshapedx = x.flatten()[indexing]

  output, maxes = quantize_rowwise(reshapedx,dt)

  return output.reshape(x.shape), maxes, indexing

def dequantize_stable_embedding(input, maxes, indexing):
  '''Takes the quantized matrices and dequantizes them by multiplying the normalized and maxes
  Then uses the indexes to place the dequantized values back into their original spots
  returns dequantized values in the right positions
  Takes in the quantized array, the array maximums, and the indexes'''
  outreshape = input.reshape(indexing.shape)

  dequant = dequantize_rowwise(outreshape, maxes).flatten()
  dequant[indexing.flatten()] = dequant.clone()
  return dequant.reshape(input.shape)

In [16]:
def quantize_dequantize_se(mat,output_path,count):
    testing, max, indexing = quantize_stable_embedding(mat, mat.shape[0],dt = True)
    np.save(f'{output_path}/layer{count}_dt_max.npy',testing)
    np.save(f'{output_path}/layer{count}_qweights.npy',max)
    np.save(f'{output_path}/layer{count}_qweights.npy',indexing)
    return dequantize_stable_embedding(testing, max, indexing)

In [11]:
DEBUG = False

In [17]:
quantized_lst = []
if not DEBUG:
    count = 0
    curr_path = 'stable_embedding' 
    for layer in [*vgg16.features,*vgg16.classifier]:
        layer.requires_grad = False
        count += 1
        try:
            if len(layer.weight.shape) == 4:
                weights = layer.weight.detach()
                print(f'1.Layer {count}')# weights shape pre-quantization: {weights.shape}\nWeights: {weights}')
                for filter in range(0, weights.shape[0]):
                    # print(f'Filter num {filter}')
                    for channel in range(0, weights.shape[1]):
                        # print(f'Channel num {channel}')
                        # print(layer.weight[filter,channel])
                        weights[filter,channel] = quantize_dequantize_se(weights[filter,channel],curr_path,count)
                        # for row in range(0,weights.shape[2]):
                        #     weights[filter,channel, row] = quantize_dequantize_dt(weights[filter,channel,row])
                        # print(f'Finish window')
                # print(f'Layer {count} weights shape post-quantization: {weights.shape}\nWeights: {weights}')
                # layer.weight = nn.parameter.Parameter(weights)
                # print(f'2.Layer {count} weights shape post-quantization: {weights.shape}\nWeights: {weights}')
                layer.weight = nn.parameter.Parameter(weights)
                quantized_lst.append(count)
                print(f'1.Layer {count} finished quantization')
            else:
                # print("##1",layer.weight)
                weights = layer.weight.detach()
                print(f'2. Layer {count}')# weights shape pre-quantization: {layer.weight.shape}\nWeights: {weights}')
                weights = quantize_dequantize_se(weights,curr_path,count)
                # for row in tqdm(range(0,weights.shape[0])):
                #     weights[row] = quantize_dequantize_dt(weights[row])
                # print("##2",weights)
                layer.weight = nn.parameter.Parameter(weights)
                # print(f'Layer {count} weights shape post-quantization: {layer.weight.shape}\nWeights: {weights}')
                # print(layer.weight)
                # print("##3",layer.weight)
                quantized_lst.append(count)
                print(f'2.Layer {count} finished quantization')

        except ( AttributeError):
            pass
else:
    count = 0
    for layer in vgg16.classifier:
        count += 1
        try:
            if len(layer.weight.shape) == 4:
                weights = layer.weight.detach()
                print(f'Layer {count} weights shape pre-quantization: {weights.shape}')
                
                for filter in range(0, layer.weight.shape[0]):
                    for channel in range(0, layer.weight.shape[1]):
                        # print(layer.weight[filter,channel])
                        weights[filter,channel] = quantize_dequantize_se(weights[filter,channel])
                print(f'Layer {count} weights shape post-quantization: {weights.shape}')
                layer.weight = nn.parameter.Parameter(weights)
            else:
                print(f'In else loop')
                print(f'Layer {count} weights shape pre-quantization: {layer.weight.shape}')
                intermediate = quantize_dequantize_se(layer.weight.detach())
                print(intermediate.shape)
                layer.weight = nn.parameter.Parameter(intermediate)
                print(f'Layer {count} weights shape post-quantization: {layer.weight.shape}')
                # print(layer.weight)
        except (TypeError, AttributeError):
            pass


1.Layer 1
1.Layer 1 finished quantization
1.Layer 3
1.Layer 3 finished quantization
1.Layer 6
1.Layer 6 finished quantization
1.Layer 8
1.Layer 8 finished quantization
1.Layer 11
1.Layer 11 finished quantization
1.Layer 13
1.Layer 13 finished quantization
1.Layer 15
1.Layer 15 finished quantization
1.Layer 18
1.Layer 18 finished quantization
1.Layer 20
1.Layer 20 finished quantization
1.Layer 22
1.Layer 22 finished quantization
1.Layer 25
1.Layer 25 finished quantization
1.Layer 27
1.Layer 27 finished quantization
1.Layer 29
1.Layer 29 finished quantization
2. Layer 32
2.Layer 32 finished quantization
2. Layer 35
2.Layer 35 finished quantization
2. Layer 36
2.Layer 36 finished quantization
2. Layer 38
2.Layer 38 finished quantization


In [18]:
model_path = f'{curr_path}/Stable_Embedding_model'
torch.save(vgg16.state_dict(), model_path)

In [46]:
# DT QUANTIZATION
if Test:
    #Testing Accuracy
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data[0].to(device), data[1].to(device)
            outputs = vgg16(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

  0%|          | 0/2500 [00:00<?, ?it/s]/Users/yanran/miniconda3/envs/hw2/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/yanran/miniconda3/envs/hw2/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /Users/yanran/miniconda3/envs/hw2/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <E459C462-F863-3A5A-AC9F-FD77B14BE845> /Users/yanran/miniconda3/envs/hw2/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/yanran/miniconda3/envs/hw2/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to

Accuracy of the network on the 10000 test images: 87.13 %
